# テキスト分析サンプルコード
このノートブックでは AWS 環境を利用してテキスト分析を体験するためのサンプルコード集です。
Amazon Comprehend、Amazon Bedrock、MeCab、Doc2Vecなどの様々な技術を使用して、日本語テキストの分析を行うサンプルコードを掲載しています。

## 概要

日本語テキストの自然言語処理において、以下の分析手法を比較・検証することを目的としています：

* **Amazon Comprehend**: AWSの自然言語処理サービスを使用したキーフレーズ抽出、エンティティ検出、感情分析
* **MeCab**: 日本語形態素解析による単語頻度分析
* **Doc2Vec**: 文書ベクトル化による類似度分析
*  **Amazon Bedrock**: Claude 4 Sonnetを使用した特徴的な単語抽出

必要なファイルを S3 からコピーする

In [ ]:
!aws s3 cp s3://text-analysis-hanson-2025/sample-text sample-text  --recursive --include "*.md"
!aws s3 cp s3://text-analysis-hanson-2025/templete templete  --recursive --include "*.md"
!aws s3 cp s3://text-analysis-hanson-2025/requirements.txt requirements.txt
!aws s3 cp s3://text-analysis-hanson-2025/MEIRYO.TTC MEIRYO.TTC

必要なライブラリをインストールする

In [ ]:
!pip install -r requirements.txt

## Amazon Comprehend
Python を使って、 Amazon Comprehend のテキスト分析を体験します

Amazon Comprehend は、以下の種類のインサイトを分析します。

* Entities — ドキュメント内での人物、場所、アイテム、場所への言及 (名前)。
* Key phrases — ドキュメントに現れるフレーズ。例えば、バスケットボールの試合に関するドキュメントでは、チームの名前、会場の名前、最終スコアを返すことができます。
* 個人を特定できる情報 (PII) — 住所や銀行口座番号、電話番号など、個人を特定できる個人データ。
* Language — ドキュメントの主要言語。
* Sentiment — ドキュメントの主要な感情。肯定的、中立、否定的、混在のいずれからにできます。
* Targeted sentiment — ドキュメント内の特定のエンティティに関連する感情。出現する各エンティティに対する感情は、肯定的、否定的、中立、混在のいずれかにできます。
* Syntax — ドキュメント内の各単語の品詞。

なお、日本語の場合は `キーフレーズの検出` ・ `エンティティの検出` ・ `感情の分析` の3つにだけ対応しています。テキストファイル（ Markdown 形式）を読み込んで Amazon Comprehend に入力し、3パターンでの分析結果を出力するようなコードになっています。


Amazon Comprehend 参考資料

* デベロッパーガイド: [Amazon Comprehend とは](https://docs.aws.amazon.com/ja_jp/comprehend/latest/dg/what-is.html)
* ハンズオン: [AWS Hands-on for Beginners: AWS Managed AI/ML サービス はじめの一歩](https://pages.awscloud.com/JAPAN-event-OE-Hands-on-for-Beginners-AIML-2022-confirmation_003.html)
* 参考になるブログ: [【AWS】【機械学習】Amazon ComprehendのCustom classification (カスタム分類) を使って日本語のテキストを感情分析してみた](https://techblog.ap-com.co.jp/entry/2024/07/01/093000)
* カスタムモデルの例: [Amazon Comprehend を使用してカスタムエンティティレコグナイザーを構築する](https://aws.amazon.com/jp/blogs/news/build-a-custom-entity-recognizer-using-amazon-comprehend/)

まずは関数を定義しておきます

In [ ]:
import boto3
import os

# --- 設定 ---
# SageMaker Notebookインスタンスから実行する場合、ロールにComprehendへのアクセス権があれば
# access_key, secret_key, region_name の設定は不要です。
# boto3が自動的に認証情報を解決します。
REGION_NAME = "ap-northeast-1" # 例: 東京リージョン


# --- 関数定義 ---
def analyze_text_with_comprehend(text, region_name):
    """
    Amazon Comprehend を使ってテキスト分析を実行する関数。
    - キーフレーズの検出
    - エンティティの検出
    - 感情の分析
    """
    try:
        # Comprehend クライアントの作成
        comprehend = boto3.client("comprehend", region_name=region_name)
        language_code = 'ja'

        # --- 1. キーフレーズ分析 ---
        print("--- 1. Amazon Comprehendによるキーフレーズ分析 ---")
        key_phrases_response = comprehend.detect_key_phrases(Text=text, LanguageCode=language_code)
        print("検出されたキーフレーズ:")
        if key_phrases_response['KeyPhrases']:
            for phrase in key_phrases_response['KeyPhrases']:
                print(f"- {phrase['Text']} (スコア: {phrase['Score']:.4f})")
        else:
            print("キーフレーズは見つかりませんでした。")

        print("\n" + "="*50 + "\n")

        # --- 2. エンティティ検出 ---
        print("--- 2. Amazon Comprehendによるエンティティ検出 ---")
        entities_response = comprehend.detect_entities(Text=text, LanguageCode=language_code)
        print("検出されたエンティティ:")
        if entities_response['Entities']:
            for entity in entities_response['Entities']:
                print(f"- {entity['Text']} (タイプ: {entity['Type']}, スコア: {entity['Score']:.4f})")
        else:
            print("エンティティは見つかりませんでした。")

        print("\n" + "="*50 + "\n")

        # --- 3. 感情分析 ---
        print("--- 3. Amazon Comprehendによる感情分析 ---")
        sentiment_response = comprehend.detect_sentiment(Text=text, LanguageCode=language_code)
        print(f"全体の感情: {sentiment_response['Sentiment']}")
        print("感情スコア:")
        for sentiment, score in sentiment_response['SentimentScore'].items():
            print(f"- {sentiment}: {score:.4f}")

    except Exception as e:
        print(f"エラーが発生しました: {e}")


print("準備OK")

`準備OK` と出力されたら、分析できる状態になっています。もしエラーが表示される場合は、内容を確認してコードを修正しましょう。

続いて `Amazon Comprehend` を使ったテキスト分析を実行します。最初は `sample-text/sample.md` を対象に分析するようになっていますが、慣れてきたら他のファイルも試してみましょう

In [ ]:
try:
    # サンプルテキストファイルを読み込む
    # 必要に応じてパスを調整してください。
    file_path = "sample-text/sample.md"
    with open(file_path, 'r', encoding='utf-8') as f:
        sample_text = f.read()

    # テキストが5000バイトを超えている場合は分割する必要があるが、今回はサンプルなので全体を一度に処理
    if len(sample_text.encode('utf-8')) > 4800: # 念のため少し余裕を持たせる
        print("警告: テキストサイズが大きいため、一部を切り詰めて分析します。")
        # UTF-8でエンコードしてからバイト数を基準に切り詰める
        encoded_text = sample_text.encode('utf-8')
        truncated_encoded_text = encoded_text[:4800]
        # 不完全なマルチバイト文字で終わらないようにデコード・再エンコード
        sample_text = truncated_encoded_text.decode('utf-8', 'ignore')

    # Comprehendで分析を実行
    analyze_text_with_comprehend(sample_text, REGION_NAME)

except FileNotFoundError:
    print(f"エラー: ファイルが見つかりません。パスを確認してください: {file_path}")
except Exception as e:
    print(f"メイン処理でエラーが発生しました: {e}")

## MeCab

次は、日本語形態素解析による単語頻度分析ができる `MeCab` を使ったテキスト分析を体験します。

In [ ]:
# うまく実行できない場合は、以下のコマンドを先頭の # を削除してから実行する
# !python -m unidic download

In [ ]:
import MeCab
import os
from collections import Counter
import re

# --- 設定 ---
# MeCabの辞書パス。`!pip install ipadic` でインストールした場合、通常は自動で解決されますが、
# 環境によっては明示的な指定が必要です。
# 例: MECAB_ARGS = "-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
# ここではデフォルトの辞書(ipadic)を想定します。
MECAB_ARGS = ""

# 分析対象のサンプルファイル
# ABSOLUTE_FILE_PATH = "/workspaces/esio/amazon-comprehend/sample-text/sample.md"

# --- 関数定義 ---

def analyze_word_frequency_with_mecab(text):
    """
    MeCabを使ってテキストの単語出現頻度を分析する関数。
    品詞フィルタリング、原型化、ストップワード除去などのベストプラクティスを適用します。
    """
    try:
        # MeCabの初期化
        tagger = MeCab.Tagger(MECAB_ARGS)
        tagger.parse('') # UnicodeDecodeErrorを避けるためのおまじない

        print("--- MeCabによる頻出単語分析（ベストプラクティス適用） ---")
        print("品詞フィルタリング、原型化、ストップワード除去などを行い、意味のある単語を抽出します。")

        node = tagger.parseToNode(text)
        words = []
        # ストップワードの定義（カタカナの原型も追加）
        stop_words = ['こと', 'もの', 'ため', 'これ', 'それ', 'あれ', '私', 'よう', 'さん', 
                      'する', 'いる', 'なる', 'ある', 'いう', 'スル', 'イル']

        while node:
            # BOS/EOS (文頭・文末) や空のノードはスキップ
            if node.surface == "":
                node = node.next
                continue

            # 品詞情報などをカンマ区切りで取得
            features = node.feature.split(',')
            
            # エラー回避: featuresの要素数が足りない場合はスキップ
            if len(features) < 7:
                node = node.next
                continue

            pos = features[0]         # 品詞
            pos_detail1 = features[1] # 品詞細分類1
            original_form = features[6] # 原型
            surface_form = node.surface # 表層形

            # 抽出対象の品詞を定義 (名詞、動詞、形容詞、副詞)
            target_pos = ['名詞', '動詞', '形容詞', '副詞']

            if pos in target_pos:
                # 品詞によって原型を使うか表層形を使うか選択
                if pos in ['動詞', '形容詞']:
                    word_to_check = original_form
                else: # 名詞、副詞など
                    word_to_check = surface_form

                # フィルタリング処理
                is_valid = True
                # 1. 除外する品詞細分類（名詞の場合）
                if pos == '名詞' and pos_detail1 in ['非自立', '代名詞', '数', '接尾', '接続詞的']:
                    is_valid = False
                # 2. 単語が'*'や空の場合は除外
                if word_to_check == '*' or not word_to_check:
                    is_valid = False
                # 3. ストップワードに含まれていれば除外
                if word_to_check in stop_words:
                    is_valid = False
                # 4. 1文字のひらがな・カタカナは除外 (漢字やアルファベットは残す)
                if len(word_to_check) == 1 and re.fullmatch(r'[ぁ-んァ-ヶ]', word_to_check):
                    is_valid = False

                if is_valid:
                    words.append(word_to_check)
            
            node = node.next
        
        word_counts = Counter(words)
        print("\n最も頻繁に出現する意味のある単語 (トップ15):")
        if word_counts:
            for word, count in word_counts.most_common(15):
                print(f"- {word}: {count}回")
        else:
            print("分析対象の単語が見つかりませんでした。")

    except RuntimeError as e:
        print(f"MeCabの実行エラー: {e}")
        print("MeCabまたは辞書が正しくインストールされていない可能性があります。")
        print("ノートブックのセルで `!pip install mecab-python3 ipadic` を実行してください。")
    except Exception as e:
        print(f"エラーが発生しました: {e}")

# --- メイン処理 ---

print("準備OK")


`準備OK` と出力されたら、テキスト分析ができる状態になっています。

次のセルを実行して、テキスト分析を体験しましょう。こちらも同様に、テキストファイル（マークダウン形式）を読み込んで分析するようになっていますので、慣れてきたら別のファイルでも試してみましょう。

In [ ]:
# 分析対象のサンプルファイル
file_path = "sample-text/sample.md"


"""テキストファイルを分析"""
try:
    with open(file_path, 'r', encoding='utf-8') as f:
        sample_text = f.read()
    
    analyze_word_frequency_with_mecab(sample_text)

except FileNotFoundError:
    print(f"エラー: ファイルが見つかりません。パスを確認してください: {file_path}")
except Exception as e:
    print(f"メイン処理でエラーが発生しました: {e}")


## doc2vec
### doc2vec とは

Doc2Vecは任意の長さの文章を固定長のベクトルに変換する技術です。
Word2Vecが単語の分散表現を獲得するものだったのに対し、Doc2Vecは文章や文書の分散表現を獲得します。文章の分散表現を獲得する手法としては古典的なものとしてはBag-of-WordsやTF-IDFがありますが、それらは下記のような弱点を有しています。

文章内の単語の語順情報を有していない
同義語でも完全に異なる独立した単語として認識する
これらはカウントベースと呼ばれる手法ですが、Doc2Vecは上記弱点を克服すべく違うアプローチで文章の分散表現の獲得を試みています。

> 引用: [(Qiita) Doc2Vecについてまとめる](https://qiita.com/g-k/items/5ea94c13281f675302ca#doc2vec%E3%81%A8%E3%81%AF%E4%BD%95%E3%81%8B)


In [ ]:
import os
import glob
import MeCab
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import numpy as np
import re
import time
from collections import Counter
import json


# --- 設定 ---
MECAB_ARGS = ""
SAMPLE_DIR_PATH = "sample-text/"    # ディレクトリ内の全ての文書を対象に分析します

# ハンズオン向け設定
MAX_FILES = 5  # 適度なファイル数
MAX_TEXT_LENGTH = 3000  # 十分な分析データ
EPOCHS = 15  # バランスの取れたエポック数
MIN_COUNT = 2
VECTOR_SIZE = 100


class Doc2VecAnalyzer:
    def __init__(self):
        self.model = None
        self.documents = []
        self.document_names = []
        self.word_stats = {}
        
    def preprocess_text(self, text, doc_name):
        """
        高品質な前処理：統計情報も収集
        """
        if len(text) > MAX_TEXT_LENGTH:
            text = text[:MAX_TEXT_LENGTH]
        
        tagger = MeCab.Tagger(MECAB_ARGS)
        tagger.parse('')
        node = tagger.parseToNode(text)
        
        words = []
        pos_stats = Counter()
        
        # 拡張されたstop wordsリスト
        stop_words = {
            # 基本的な機能語
            'こと', 'もの', 'ため', 'これ', 'それ', 'あれ', 'どれ', 'よう', 'さん', 'ところ',
            'とき', 'とこ', 'など', 'なに', 'なん', 'どこ', 'いつ', 'だれ', 'どう', 'なぜ',
            
            # 代名詞・指示語
            '私', '僕', '俺', '君', '彼', '彼女', 'あなた', 'みなさん', 'みんな',
            'ここ', 'そこ', 'あそこ', 'どこか', 'いま', 'いつか', 'どこでも',
            
            # 助詞的な名詞
            '上', '下', '中', '前', '後', '左', '右', '横', '隣', '間', '内', '外',
            '先', '奥', '手前', '向こう', '以上', '以下', '未満', '程度', '以外',
            
            # 時間・頻度表現
            '今日', '昨日', '明日', '今年', '去年', '来年', '今月', '先月', '来月',
            '毎日', '毎回', '毎年', '毎月', '毎週', '常に', 'いつも', 'たまに',
            
            # 数量・程度表現
            '全て', '全部', 'すべて', '一部', '半分', '大部分', '少し', 'ちょっと',
            'かなり', 'とても', 'すごく', 'めちゃくちゃ', '非常', '極めて',
            
            # 接続・転換表現
            'しかし', 'でも', 'だが', 'ただし', 'ところが', 'けれど', 'けれども',
            'そして', 'また', 'さらに', 'それから', 'それで', 'そこで', 'つまり',
            
            # 感嘆・応答表現
            'はい', 'いいえ', 'ええ', 'うん', 'そう', 'そうです', 'なるほど',
            'おお', 'ああ', 'うーん', 'えー', 'まあ', 'やはり', 'やっぱり',
            
            # 一般的すぎる動詞・形容詞の語幹
            'する', 'なる', 'ある', 'いる', 'できる', 'みる', 'いく', 'くる',
            'いい', 'よい', '悪い', '大きい', '小さい', '新しい', '古い',
            
            # ビジネス・技術文書でよく出る一般語
            '場合', '状況', '状態', '方法', '手段', '方式', '形式', '種類', '方向',
            '結果', '効果', '影響', '関係', '関連', '対象', '目的', '理由', '原因',
            '問題', '課題', '解決', '改善', '向上', '発展', '進歩', '変化', '変更',
            
            # 単位・助数詞的表現
            '個', '本', '枚', '台', '人', '回', '度', '倍', '割', 'パーセント',
            '時間', '分', '秒', '日', '週間', 'ヶ月', '年間', 'メートル', 'キロ'
        }

        while node:
            if not node.surface:
                node = node.next
                continue

            features = node.feature.split(',')
            if len(features) < 7:
                node = node.next
                continue

            pos = features[0]
            pos_detail1 = features[1]
            original_form = features[6]
            surface_form = node.surface

            # 統計収集
            pos_stats[pos] += 1

            # 高品質な品詞選択
            target_pos = ['名詞', '動詞', '形容詞']
            if pos in target_pos:
                word_to_check = original_form if pos in ['動詞', '形容詞'] else surface_form
                
                # 高品質フィルタリング
                if (word_to_check != '*' and 
                    word_to_check not in stop_words and 
                    len(word_to_check) > 1 and
                    not re.match(r'^[0-9]+$', word_to_check) and  # 数字のみ除外
                    pos != '名詞' or pos_detail1 not in ['数', '非自立', '代名詞']):
                    words.append(word_to_check)

            node = node.next
        
        # 文書統計を保存
        self.word_stats[doc_name] = {
            'total_words': len(words),
            'unique_words': len(set(words)),
            'pos_distribution': dict(pos_stats.most_common(5))
        }
        
        return words

    def train_model(self, documents, document_names):
        """
        Doc2Vecモデルの学習
        """
        print("=== Doc2Vec モデル学習フェーズ ===")
        
        tagged_documents = [
            TaggedDocument(doc, [name]) 
            for doc, name in zip(documents, document_names)
        ]
        
        self.model = Doc2Vec(
            vector_size=VECTOR_SIZE,
            min_count=MIN_COUNT,
            epochs=EPOCHS,
            workers=min(4, os.cpu_count()),
            dm=1,  # PV-DM (分散メモリ)
            window=5,
            alpha=0.025,
            min_alpha=0.00025
        )
        
        start_time = time.time()
        self.model.build_vocab(tagged_documents)
        
        print(f"学習開始: {len(documents)}文書, 語彙数: {len(self.model.wv.key_to_index)}")
        self.model.train(tagged_documents, total_examples=self.model.corpus_count, epochs=self.model.epochs)
        
        training_time = time.time() - start_time
        print(f"学習完了 (所要時間: {training_time:.2f}秒)")
        
        return self.model

    def analyze_document_similarity(self):
        """
        文書間類似度分析
        """
        print("\n=== 文書間類似度分析 ===")
        
        if len(self.document_names) < 2:
            print("類似度分析には2つ以上の文書が必要です。")
            return
        
        similarities = []
        for i, doc1 in enumerate(self.document_names):
            for j, doc2 in enumerate(self.document_names[i+1:], i+1):
                try:
                    similarity = self.model.dv.similarity(doc1, doc2)
                    similarities.append((doc1, doc2, similarity))
                    print(f"{os.path.basename(doc1)} ⟷ {os.path.basename(doc2)}: {similarity:.4f}")
                except KeyError as e:
                    print(f"文書ベクトルが見つかりません: {e}")
        
        # 最も類似した文書ペア
        if similarities:
            most_similar = max(similarities, key=lambda x: x[2])
            print(f"\n最も類似した文書ペア:")
            print(f"  {os.path.basename(most_similar[0])} ⟷ {os.path.basename(most_similar[1])}")
            print(f"  類似度: {most_similar[2]:.4f}")

    def analyze_word_similarity(self):
        """
        単語類似度分析（教育的な解説付き）
        """
        print("\n=== 単語類似度分析 ===")
        
        # 語彙から頻出単語を選択
        word_freq = Counter()
        for doc in self.documents:
            word_freq.update(doc)
        
        common_words = [word for word, freq in word_freq.most_common(20) 
                       if word in self.model.wv.key_to_index]
        
        if not common_words:
            print("分析可能な単語が見つかりません。")
            return
        
        print(f"頻出単語から分析対象を選択: {common_words[:5]}")
        
        for word in common_words[:3]:
            print(f"\n'{word}' の類似単語:")
            try:
                similar_words = self.model.wv.most_similar(word, topn=5)
                for i, (sim_word, similarity) in enumerate(similar_words, 1):
                    print(f"  {i}. {sim_word} (類似度: {similarity:.4f})")
            except KeyError:
                print(f"  '{word}' の類似単語を見つけられませんでした。")

    def analyze_word_clusters(self):
        """
        単語クラスタリング分析
        """
        print("\n=== 単語クラスタリング分析 ===")
        
        # 頻出単語のベクトルを取得
        word_freq = Counter()
        for doc in self.documents:
            word_freq.update(doc)
        
        target_words = [word for word, freq in word_freq.most_common(15) 
                       if word in self.model.wv.key_to_index]
        
        if len(target_words) < 3:
            print("クラスタリングに十分な単語がありません。")
            return
        
        print(f"分析対象単語: {target_words}")
        
        # 簡易クラスタリング（類似度ベース）
        clusters = {}
        processed = set()
        
        for word in target_words:
            if word in processed:
                continue
                
            cluster = [word]
            processed.add(word)
            
            try:
                similar_words = self.model.wv.most_similar(word, topn=10)
                for sim_word, similarity in similar_words:
                    if sim_word in target_words and similarity > 0.3 and sim_word not in processed:
                        cluster.append(sim_word)
                        processed.add(sim_word)
                
                if len(cluster) > 1:
                    clusters[f"クラスタ_{len(clusters)+1}"] = cluster
            except KeyError:
                continue
        
        print("\n単語クラスタ:")
        for cluster_name, words in clusters.items():
            print(f"  {cluster_name}: {', '.join(words)}")

    def generate_analysis_report(self):
        """
        分析レポートの生成
        """
        print("\n" + "="*60)
        print("=== Doc2Vec 分析レポート ===")
        print("="*60)
        
        print(f"\n【モデル情報】")
        print(f"  ベクトル次元数: {self.model.vector_size}")
        print(f"  語彙数: {len(self.model.wv.key_to_index)}")
        print(f"  学習エポック数: {self.model.epochs}")
        
        print(f"\n【文書統計】")
        for doc_name, stats in self.word_stats.items():
            print(f"  {os.path.basename(doc_name)}:")
            print(f"    総単語数: {stats['total_words']}")
            print(f"    ユニーク単語数: {stats['unique_words']}")
            print(f"    語彙多様性: {stats['unique_words']/stats['total_words']:.3f}")
        
        print(f"\n【頻出単語トップ10】")
        word_freq = Counter()
        for doc in self.documents:
            word_freq.update(doc)
        
        for i, (word, freq) in enumerate(word_freq.most_common(10), 1):
            print(f"  {i:2d}. {word} ({freq}回)")


print("準備OK")

In [ ]:
def main():
    """
    メイン処理
    """
    analyzer = Doc2VecAnalyzer()

    try:
        start_time = time.time()

        # ファイル読み込み
        search_path = os.path.join(SAMPLE_DIR_PATH, '*.md')
        file_paths = glob.glob(search_path)

        if not file_paths:
            print(f"エラー: サンプル文書が見つかりません: {SAMPLE_DIR_PATH}")
            return

        # ファイル数制限
        limited_files = file_paths[:MAX_FILES]
        print(f"=== ファイル読み込み ===")
        print(f"対象ファイル数: {len(limited_files)}")

        documents = []
        document_names = []

        for i, path in enumerate(limited_files):
            filename = os.path.basename(path)
            print(f"処理中: {filename} ({i+1}/{len(limited_files)})")

            with open(path, 'r', encoding='utf-8') as f:
                text = f.read()

            words = analyzer.preprocess_text(text, path)
            if words:
                documents.append(words)
                document_names.append(path)
                print(f"  抽出単語数: {len(words)}")

        if not documents:
            print("エラー: 分析可能な文書がありません。")
            return

        analyzer.documents = documents
        analyzer.document_names = document_names

        # モデル学習
        analyzer.train_model(documents, document_names)

        # 各種分析実行
        analyzer.analyze_document_similarity()
        analyzer.analyze_word_similarity()
        analyzer.analyze_word_clusters()
        analyzer.generate_analysis_report()

        total_time = time.time() - start_time
        print(f"\n総処理時間: {total_time:.2f}秒")

    except Exception as e:
        print(f"エラーが発生しました: {e}")
        import traceback
        traceback.print_exc()


if __name__ == '__main__':
    main()


# 生成AIを使用する場合
ここからは、生成AIを使用した自然言語処理のサンプルコードです。

[Amazon Bedrock](https://aws.amazon.com/jp/bedrock/) を使用すると、様々な生成AIモデルを利用することができます。今回は、 `Anthropic` が提供するテキスト系の LLM である `Claude 4 Sonnet` を利用して、テキスト分析を実施します。

分析のステップは以下のとおりです。

1. LLM で特徴的な単語を抽出してリストとして取得する
2. LLM のレスポンスに不要なテキストが含まれる可能性があるので、必要なレスポンス(json)だけをパースする。
3. リストの特徴後を1つずつ、元のテキスト内でカウントする。 >>> 結果: `analysis_result.json` 
4. (おまけ) 出現回数が多いワードを、ワードクラウドで可視化する >>> 結果: `wordcloud_result.png`

(参考)
LLM 用のプロンプトは `templete/llm.md` にあります。 GenU のビルダーモードでも利用できるように作成されています。

In [ ]:
import boto3
import json
import re
from datetime import datetime
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def extract_json_from_text(text):
    """
    テキストからJSONを抽出する関数
    LLMの出力にJSON以外のテキストが含まれている場合に対応
    """
    # 複数のパターンでJSONを探す
    patterns = [
        # パターン1: 最初の{から最後の}まで
        r'\{.*\}',
        # パターン2: ```json ブロック内
        r'```json\s*(\{.*?\})\s*```',
        # パターン3: ``` ブロック内
        r'```\s*(\{.*?\})\s*```'
    ]
    
    for pattern in patterns:
        matches = re.findall(pattern, text, re.DOTALL)
        for match in matches:
            try:
                # JSONとして解析を試行
                parsed = json.loads(match)
                return parsed
            except json.JSONDecodeError:
                continue
    
    # 行ごとに分割してJSONらしい行を探す
    lines = text.split('\n')
    json_lines = []
    in_json = False
    brace_count = 0
    
    for line in lines:
        stripped = line.strip()
        if stripped.startswith('{'):
            in_json = True
            json_lines = [line]
            brace_count = line.count('{') - line.count('}')
        elif in_json:
            json_lines.append(line)
            brace_count += line.count('{') - line.count('}')
            if brace_count <= 0:
                break
    
    if json_lines:
        try:
            json_text = '\n'.join(json_lines)
            return json.loads(json_text)
        except json.JSONDecodeError:
            pass
    
    return None

def create_wordcloud(word_frequency, top_n=10):
    """
    単語の出現回数からワードクラウドを生成する関数
    """
    # 上位N位までの単語を取得
    top_words = dict(list(word_frequency.items())[:top_n])
    
    if not top_words:
        print("❌ ワードクラウド用のデータがありません")
        return
    
    print(f"🎨 上位{top_n}位までの単語でワードクラウドを生成中...")
    
    try:
        # 日本語フォントファイルのパスを指定
        font_path = "./MEIRYO.TTC"
        
        # ワードクラウドを生成
        wordcloud = WordCloud(
            width=800,
            height=400,
            background_color='white',
            max_words=top_n,
            relative_scaling=0.5,
            colormap='viridis',
            font_path=font_path  # MEIRYOフォントを指定
        ).generate_from_frequencies(top_words)
        
        # プロット設定（日本語フォント対応）
        plt.rcParams['font.family'] = 'DejaVu Sans'  # 英語部分用
        plt.figure(figsize=(10, 5))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.title(f'Word Frequency Ranking Top {top_n}', fontsize=16, pad=20)
        
        # ファイルに保存
        output_file = "wordcloud_result.png"
        plt.savefig(output_file, bbox_inches='tight', dpi=300)
        print(f"✅ ワードクラウドを {output_file} に保存しました")
        
        # 画面に表示
        plt.show()
        
    except FileNotFoundError:
        print("❌ MEIRYO.TTCフォントファイルが見つかりません")
        print("💡 カレントディレクトリにMEIRYO.TTCファイルがあることを確認してください")
    except Exception as e:
        print(f"❌ ワードクラウド生成でエラーが発生しました: {e}")


print("準備OK")

In [ ]:
def analyze_text_with_bedrock():
    """
    メイン分析関数：テキストファイルを読み込み、Bedrockで分析し、結果を出力
    """
    
    # 1. 設定
    # Claude 4 Sonnet用の推論プロファイルIDを使用
    MODEL_ID = "us.anthropic.claude-sonnet-4-20250514-v1:0"
    REGION = "us-east-1"
    
    print("🚀 Amazon Bedrock テキスト分析を開始します...")
    
    # 2. テキストファイルを読み込み
    try:
        with open("sample-text/sample.md", "r", encoding="utf-8") as file:
            sample_text = file.read()
        print("✅ サンプルテキストを読み込みました")
    except FileNotFoundError:
        print("❌ sample.mdファイルが見つかりません")
        return
    
    # 3. プロンプトテンプレートを読み込み
    try:
        with open("templete/llm.md", "r", encoding="utf-8") as file:
            prompt_template = file.read()
        print("✅ プロンプトテンプレートを読み込みました")
    except FileNotFoundError:
        print("❌ llm.mdファイルが見つかりません")
        return
    
    # 4. Bedrockクライアントを作成
    bedrock_runtime = boto3.client(
        service_name="bedrock-runtime",
        region_name=REGION
    )
    print("✅ Bedrockクライアントを初期化しました")
    
    # 5. プロンプトを作成（テンプレートにテキストを埋め込み）
    prompt = prompt_template.replace("{{解析対象のテキストをLLMに渡す}}", sample_text)
    
    # 6. Bedrock converse_stream APIを呼び出し
    try:
        print("🤖 Claude 4 Sonnet で分析中...")
        
        response = bedrock_runtime.converse_stream(
            modelId=MODEL_ID,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "text": prompt
                        }
                    ]
                }
            ],
            inferenceConfig={
                "maxTokens": 2000,
                "temperature": 0.1
            }
        )
        
        # ストリーミングレスポンスを収集
        llm_output = ""
        for event in response["stream"]:
            if "contentBlockDelta" in event:
                delta = event["contentBlockDelta"]["delta"]
                if "text" in delta:
                    llm_output += delta["text"]
        
        print("✅ LLMからの応答を取得しました")
        print(f"📄 LLM出力プレビュー: {llm_output[:200]}...")
        
    except Exception as e:
        print(f"❌ Bedrock API呼び出しでエラーが発生しました: {e}")
        return
    
    # 7. LLMの出力からJSONを抽出（堅牢なパース処理）
    try:
        print("🔍 JSON形式のデータを抽出中...")
        
        # 堅牢なJSON抽出を実行
        keywords_data = extract_json_from_text(llm_output)
        
        if keywords_data is None:
            print("❌ 有効なJSONが見つかりませんでした")
            print(f"LLM完全出力:\n{llm_output}")
            return
        
        # resultsキーから単語配列を取得
        if "results" not in keywords_data:
            print("❌ 'results'キーが見つかりません")
            print(f"取得したJSON: {keywords_data}")
            return
            
        keywords = keywords_data["results"]
        print(f"✅ {len(keywords)}個の特徴的な単語を抽出しました")
        print(f"📝 抽出された単語: {keywords}")
        
    except Exception as e:
        print(f"❌ JSON解析でエラーが発生しました: {e}")
        print(f"LLM完全出力:\n{llm_output}")
        return
    
    # 8. 各単語の出現回数をカウント
    print("🔍 単語の出現回数をカウント中...")
    
    word_count = {}
    for keyword in keywords:
        # 正規表現で単語の出現回数をカウント（大文字小文字を区別しない）
        pattern = re.escape(keyword)
        matches = re.findall(pattern, sample_text, re.IGNORECASE)
        count = len(matches)
        word_count[keyword] = count
        print(f"  📝 '{keyword}': {count}回")
    
    # 9. 出現回数の多い順に並び替え
    print("📊 出現回数の多い順に並び替え中...")
    sorted_word_count = dict(sorted(word_count.items(), key=lambda x: x[1], reverse=True))
    
    print("🏆 出現回数ランキング:")
    for i, (word, count) in enumerate(sorted_word_count.items(), 1):
        print(f"  {i}位: '{word}' - {count}回")
    
    # 10. 結果をJSON形式で出力
    result = {
        "analysis_timestamp": datetime.now().isoformat(),
        "source_file": "sample-text/sample.md",
        "model_used": MODEL_ID,
        "extracted_keywords": keywords,
        "word_frequency": sorted_word_count,  # 並び替え済みの辞書を使用
        "word_frequency_ranking": [
            {"rank": i, "word": word, "count": count} 
            for i, (word, count) in enumerate(sorted_word_count.items(), 1)
        ],
        "total_words_analyzed": len(keywords)
    }
    
    print("\n" + "="*50)
    print("📊 分析結果（JSON形式）")
    print("="*50)
    print(json.dumps(result, ensure_ascii=False, indent=2))
    
    # 11. 結果をファイルに保存
    output_file = "analysis_result.json"
    with open(output_file, "w", encoding="utf-8") as file:
        json.dump(result, file, ensure_ascii=False, indent=2)
    
    print(f"\n✅ 結果を {output_file} に保存しました")
    
    # 12. ワードクラウドを生成
    create_wordcloud(sorted_word_count, top_n=10)
    
    print("🎉 分析完了！")

if __name__ == "__main__":
    analyze_text_with_bedrock()
